### scikit-mobility tutorials

# 4 - Flow models

Generative models of collective spatial flows.

- [Gravity](#Gravity)
    - [Singly Constrained](#Singly-constrained)
    - [Globally Constrained](#Globally-constrained)
- [Radiation](#Radiation)
- [Population Weighted Opportunities PWO](#Population-Weighted-Opportunities-PWO)
- [Rank Distance](#Rank-Distance)

In [2]:
import skmob
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [3]:
from skmob.utils.gislib import load_spatial_tessellation

In [4]:
st = load_spatial_tessellation('../data_test/test_flow_models/location2info_rome', positive_relevance=False)
len(st)

1358

In [5]:
st[0]

{'lat': 41.66150096176846, 'lng': 12.41920512671709, 'relevance': 0}

In [6]:
rome_flows = pd.read_csv('../data_test/test_flow_models/rome_flows.csv')
print(len(rome_flows))
rome_flows.head()

1507984


,origin,destination,flow
0,0,0,260
1,0,1,28
2,0,2,3
3,0,3,11
4,0,4,1


## Gravity

### Singly constrained

In [7]:
from skmob.models.migration import Gravity

In [8]:
gravity_singly = Gravity(gravity_type='singly constrained')
print(gravity_singly)

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.0], origin_exp=1.0, destination_exp=1.0, gravity_type="singly constrained")


In [9]:
gravity_singly.fit(st, rome_flows.head(1000))
print(gravity_singly)

100%|██████████| 1000/1000 [00:00<00:00, 5993.98it/s]


Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-0.46540196266829603], origin_exp=1.0, destination_exp=0.03017564316349213, gravity_type="singly constrained")


In [10]:
flows_singly = gravity_singly.generate(st, out_format='flows')
flows_singly

100%|██████████| 1358/1358 [00:01<00:00, 776.18it/s] 
/data2/scikit-mobility-library/skmob/models/migration.py:41: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
/data2/scikit-mobility-library/skmob/models/migration.py:187: RuntimeWarning: invalid value encountered in true_divide
  trip_probs_matrix = np.transpose(trip_probs_matrix / np.sum(trip_probs_matrix, axis=1))


ValueError: sum(pvals[:-1]) > 1.0

In [11]:
probs_singly = gravity_singly.generate(st, out_format='probabilities')
probs_singly

100%|██████████| 1358/1358 [00:01<00:00, 774.05it/s] 


ValueError: sum(pvals[:-1]) > 1.0

### Globally constrained

In [12]:
gravity_globally = Gravity(gravity_type='globally constrained')
print(gravity_globally)

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.0], origin_exp=1.0, destination_exp=1.0, gravity_type="globally constrained")


In [13]:
gravity_globally.fit(st, rome_flows.head(1000))
print(gravity_globally)

  0%|          | 0/1000 [00:00<?, ?it/s]/data2/scikit-mobility-library/skmob/models/migration.py:226: RuntimeWarning: divide by zero encountered in log
  sc_vars = [np.log(weight_origin)]
100%|██████████| 1000/1000 [00:00<00:00, 8921.00it/s]
/data2/scikit-mobility-library/skmob_dev/lib/python3.5/site-packages/numpy-1.16.0rc2-py3.5-linux-x86_64.egg/numpy/core/_methods.py:155: RuntimeWarning: invalid value encountered in subtract
  out


MissingDataError: exog contains inf or nans

In [14]:
flows_globally = gravity_globally.generate(st, out_format='flows')
flows_globally

100%|██████████| 1358/1358 [00:01<00:00, 775.74it/s] 


array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,  561,  329],
       [   0,    0,    0, ...,  550,    0, 1251],
       [   0,    0,    0, ...,  312, 1268,    0]])

In [15]:
probs_globally = gravity_globally.generate(st, out_format='probabilities')
probs_globally

100%|██████████| 1358/1358 [00:01<00:00, 775.73it/s] 


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.56831156e-04, 9.11241667e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.56831156e-04, 0.00000000e+00, 3.72196717e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.11241667e-05, 3.72196717e-04, 0.00000000e+00]])

## Radiation

In [16]:
from skmob.models.migration import Radiation

In [17]:
radiation = Radiation()
flows_avg = radiation.generate(st, out_format='flows_average')
flows_smp = radiation.generate(st, out_format='flows_sample')

100%|██████████| 1358/1358 [00:02<00:00, 501.11it/s]


In [18]:
flows_avg

array([[   8.,    9.,    0.],
       [   8.,    7.,    0.],
       [   8.,  306.,    0.],
       ...,
       [1357.,  180.,    0.],
       [1357.,  178.,    0.],
       [1357.,  181.,    0.]])

In [19]:
flows_smp

array([[   8.,    9.,    0.],
       [   8.,    7.,    0.],
       [   8.,  306.,    0.],
       ...,
       [1357.,  180.,    0.],
       [1357.,  178.,    0.],
       [1357.,  181.,    0.]])

## Population Weighted Opportunities PWO

In [ ]:
PopulationWeightedOpportunities

## Rank Distance

In [ ]:
RankDistance